In [ ]:
# !pip install networkx
# !pip install matplotlib
# !pip install tqdm
# !pip install pandas
# !pip install numpy
# !pip install graphviz
# !pip install scikit-learn

In [ ]:
import random
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations, groupby
import pandas as pd

from networkx.algorithms import tree
from networkx.algorithms import bellman_ford_predecessor_and_distance
from networkx.algorithms import floyd_warshall_predecessor_and_distance

import numpy.typing as npt

import heapq
from tqdm import tqdm

import time

# Task 1. Algorithm's analysis

## Generating graph

In [ ]:

# You can use this function to generate a random graph with 'num_of_nodes' nodes
# and 'completeness' probability of an edge between any two nodes
# If 'directed' is True, the graph will be directed
# If 'draw' is True, the graph will be drawn
def gnp_random_connected_graph(num_of_nodes: int,
                               completeness: int,
                               directed: bool = False,
                               draw: bool = False):
    """
    Generates a random graph, similarly to an Erdős-Rényi 
    graph, but enforcing that the resulting graph is conneted (in case of undirected graphs)
    """

    
    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    edges = combinations(range(num_of_nodes), 2)
    G.add_nodes_from(range(num_of_nodes))
    
    for _, node_edges in groupby(edges, key = lambda x: x[0]):
        node_edges = list(node_edges)
        random_edge = random.choice(node_edges)
        if random.random() < 0.5:
            random_edge = random_edge[::-1]
        G.add_edge(*random_edge)
        for e in node_edges:
            if random.random() < completeness:
                G.add_edge(*e)
                
    for (u,v,w) in G.edges(data=True):
        w['weight'] = random.randint(-5, 20)
                
    if draw: 
        plt.figure(figsize=(10,6))
        if directed:
            # draw with edge weights
            pos = nx.arf_layout(G)
            nx.draw(G,pos, node_color='lightblue', 
                    with_labels=True,
                    node_size=500, 
                    arrowsize=20, 
                    arrows=True)
            labels = nx.get_edge_attributes(G,'weight')
            nx.draw_networkx_edge_labels(G, pos,edge_labels=labels)
            
        else:
            nx.draw(G, node_color='lightblue', 
                with_labels=True, 
                node_size=500)
        
    return G

In [ ]:
G = gnp_random_connected_graph(20, 0.5, False, False)

## Subtask 1.1

Дослідження алгоритмів Крускала та Прима. Алгоритми будують каркаси простого зваженого графа

### Kruskal's algorithm

In [ ]:
mstk = tree.minimum_spanning_tree(G, algorithm="kruskal")

In [ ]:
nx.draw(mstk, node_color='lightblue', 
        with_labels=True, 
        node_size=500)

In [ ]:
len(mstk.edges()), sum(el[2] for el in mstk.edges.data
('weight'))

Алгоритм Крускала. Алгоритм сортує всі ребра за вагою, обирає найменше та додає до каркасу,
перевіривши чи не утвориться циклів. Алгоритм має часову складність O(E*log(V)), де E - кількість
 ребер, V -  кількість вершин

In [ ]:
def kruskals_alg(G: nx.Graph) -> nx.Graph:
    """Find the minimum spanning tree of a given graph using Kruskal's algorithm"""
    min_span_tree = nx.Graph()
    forest = []
    for v in G.nodes():
        forest.append({v})

    for u, v, w in sorted(G.edges.data('weight'), key=lambda x: x[2]):
        for i, s in enumerate(forest):
            if u in s:
                break
        for j, s in enumerate(forest):
            if v in s:
                break
        if i != j:
            min_span_tree.add_edge(u, v, weight=w)
            forest[i] |= forest[j]
            forest.pop(j)
    return nx.Graph(min_span_tree)

In [ ]:
min_span_tree = kruskals_alg(G)

In [ ]:
nx.draw(min_span_tree, node_color='lightblue',
        with_labels=True,
        node_size=200)

In [ ]:
len(min_span_tree.edges()), sum(el[2] for el in min_span_tree.edges.data
('weight'))

### Prim's algorithm

In [ ]:
mstp = tree.minimum_spanning_tree(G, algorithm="prim")

In [ ]:
nx.draw(mstp, node_color='lightblue', 
        with_labels=True, 
        node_size=500)

In [ ]:
len(mstp.edges()), sum(el[2] for el in mstp.edges.data('weight'))

Алгоритм Прима. Вибравши довільну першу вершину, алгоритм додає нові інцидентні ребра до вже
доданих вершин з найменшою вагою. Складність алгоритму O(V^2) де V - кількість вершин, проте якщо
 використовувати binary heap queue, то складність стає O(E*log(V)) де E - кількість ребер, отже
 складність така ж як у алгоритму Крускала

In [ ]:
def prims_alg(G: nx.Graph) -> nx.Graph:
    """Find the minimum spanning tree of a given graph using Prim's algorithm"""
    min_span_tree = nx.Graph()
    heap = []
    visited = set()
    for v, w in G[0].items():
        heapq.heappush(heap, (w['weight'], 0, v))
    while heap:
        w, v, u = heapq.heappop(heap)
        if u in visited:
            continue
        min_span_tree.add_edge(v, u, weight=w)
        visited.add(u)

        for neighbor in G.neighbors(u):
            if neighbor not in visited:
                heapq.heappush(heap, (G[u][neighbor]['weight'], u, neighbor))
    return min_span_tree

In [ ]:
tree = prims_alg(G)

In [ ]:
nx.draw(tree, node_color='lightblue',
        with_labels=True,
        node_size=500)

In [ ]:
len(tree.edges()), sum(el[2] for el in tree.edges.data('weight'))

In [ ]:
def measure_time(graphs: list[int], iter: int) -> tuple[list]:
    """Measure time of all algs"""
    times = [[0]*7,[0]*7,[0]*7,[0]*7]
    for i, num_of_nodes in enumerate(graphs):
        for _ in tqdm(range(iter)):
            graph = gnp_random_connected_graph(num_of_nodes, 0.8, False, False)

            start = time.perf_counter()
            kruskals_alg(graph)
            times[0][i] += time.perf_counter() - start

            start = time.perf_counter()
            prims_alg(graph)
            times[1][i] += time.perf_counter() - start

            start = time.perf_counter()
            tree.minimum_spanning_tree(graph, algorithm="kruskal")
            times[2][i] += time.perf_counter() - start

            start = time.perf_counter()
            tree.minimum_spanning_tree(graph, algorithm="prim")
            times[3][i] += time.perf_counter() - start

    return [[el/iter for el in el_time] for el_time in times]


In [ ]:
x = [10, 20, 50, 100, 200, 500, 1000]
y1, y2, y3, y4 = measure_time(x, 100)
plt.plot(x, y1, label='Kruskal\'s')
plt.plot(x, y2, label='Prim\'s')
plt.plot(x, y3, label='Built-in Kruskal\'s')
plt.plot(x, y4, label='Built-in Prim\'s')
plt.xlabel('Number of nodes')
plt.ylabel('Time (seconds)')
plt.legend()

На графіках видно як час алгоритмів зростає з кількістю вершин. Причому цікаво те, що алгоритми
вбудовані у бібліотеці networkx на великих графах працюють повільніше ніж ті що були
імплементовані мною.

## Subtask 1.2

In [ ]:
G = gnp_random_connected_graph(10, 0.5, True, True)

### Bellman-Ford algorithm

In [ ]:
# pred is a dictionary of predecessors, dist is a dictionary of distances
try:
    pred, dist = bellman_ford_predecessor_and_distance(G, 0)
    for k, v in dist.items():
        print(f"Distance to {k}:", v)
except:
    print("Negative cycle detected")

*put your code below* (delete this)

### Floyd-Warshall algorithm

In [ ]:
# pred is a dictionary of predecessors, dist is a dictionary of distances dictionaries
try:
    pred, dist = floyd_warshall_predecessor_and_distance(G) 
    for k, v in dist.items():
        print(f"Distances with {k} source:", dict(v))
except:
    print("Negative cycle detected")

*put your code below* (delete this)

---

## Some useful explanations
### How to get list of edges for your algorithm

In [ ]:
edges = list(G.edges()) # by default G.edges are EdgesView class

In [ ]:
edges[:5]

### To get edges with weights

In [ ]:
edges = list(G.edges(data=True))

In [ ]:
edges[:5]

In [ ]:
nodes = list(G.nodes())
print(nodes)

## Example on time measuring

Read more on this: https://realpython.com/python-timer/

Recall that you should measure times for 5, 10, 20, 50, 100, 200, 500 nodes 1000 times (and take mean of time taken for each node amount).

Then you should build the plot for two algorithms (x - data size, y - mean time of execution).

In [ ]:
import time
from tqdm import tqdm

In [ ]:
NUM_OF_ITERATIONS = 1000
time_taken = 0
for i in tqdm(range(NUM_OF_ITERATIONS)):
    
    # note that we should not measure time of graph creation
    G = gnp_random_connected_graph(100, 0.4, False)
    
    start = time.time()
    tree.minimum_spanning_tree(G, algorithm="prim")
    end = time.time()
    
    time_taken += end - start

time_taken / NUM_OF_ITERATIONS

## Task 2. Decision Tree Classifier 

In [53]:
# scikit-learn package
from sklearn.datasets import load_iris, load_breast_cancer, load_diabetes
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

### General idea

#### You are expected to write a quite simple, yet good core logic of decision tree classifier class. Additionaly, you need to test your results and write down a report on what you've done, which principles used and explain the general process.

#### Hopefully, you have already learned what is decision tree classifier and how it work. For better understanding, and in case if something is still unclear for you, here are some useful links on basics of DTC:
- https://www.youtube.com/watch?v=ZVR2Way4nwQ
- https://towardsdatascience.com/decision-tree-classifier-explained-a-visual-guide-with-code-examples-for-beginners-7c863f06a71e
- https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775
- https://www.kaggle.com/code/prashant111/decision-tree-classifier-tutorial
- https://towardsdatascience.com/decision-tree-classifier-explained-in-real-life-picking-a-vacation-destination-6226b2b6057

#### Also, for those interested to learn more about machine learning and particulary Desicion Trees - here is a great course on Coursera (you may be interested in the whole course or just this particular week):
- https://www.coursera.org/learn/advanced-learning-algorithms/home/week/4


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)


### Dataset
#### You can use Iris dataset for this task. It is a very popular dataset for machine learning and data science. It contains 150 samples of 3 different species of Iris flowers (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters. 
Read more on this: https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html
https://en.wikipedia.org/wiki/Iris_flower_data_set
#### However, using more interesting and intricate datasets is much appreciated. You can use any dataset you want, but it should be a classification one. For example you can use breast cancer or wine datasets, which are also available in sklearn.datasets. Or you can use any other dataset you find interesting.
P.S. In case you are not sure if your dataset is suitable, feel free to ask assistants :).

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

In [44]:
# Load dataset
iris = load_iris()
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [ ]:
iris.data.shape

This means that we have 150 entries (samples, infos about a flower). The columns being: Sepal Length, Sepal Width, Petal Length and Petal Width(features). Let's look at first two entries:

In [ ]:
iris.data[0:2]

### To undestand data little bit better, let's plot some features

In [ ]:
X = iris.data[:, :2]  # we only take the first two features.
y = iris.target

plt.figure(2, figsize=(8, 6))
plt.clf()

# Plot the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1, edgecolor="k")
plt.xlabel("Sepal length")
plt.ylabel("Sepal width")

From this we can clearly see, that even basing on those two parameters, we can clearly divide (classify) out data into several groups. For this, we will use decision tree classifier: https://scikit-learn.org/stable/modules/tree.html#tree

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

### Example of usage


**Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression**. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation.

In [45]:
clf = DecisionTreeClassifier()

X, y = iris.data, iris.target
X.shape, y.shape

((150, 4), (150,))

### Train / test split

We train our model using training dataset and evaluate its performance basing on the test dataset. Reason to use two separate datasets is that our model learns its parameters from data, thus test set allows us to check its possibilities on completely new data.

In [ ]:
X, X_test, y, y_test = train_test_split(X, y, test_size= 0.20)
X_test.shape, y_test.shape

### Model learning

It learns its parameters (where it should split data and for what threshold value) basing on the training dataset. It is done by minimizing some cost function (e.g. Gini impurity or entropy).

In [ ]:
clf = clf.fit(X, y)

### Visualization of produced tree

You do not need to understand this piece of code :)

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris")

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None, 
                     feature_names=iris.feature_names,  
                     class_names=iris.target_names,  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

### Prediction step

Now we can use our model to predict which type has a flower, basing on its parameters.

This is conducted basically via traversing the tree that you can see above.

In [ ]:
predictions = clf.predict(X_test)

### We can also measure the accuracy of our model

In [ ]:
sum(predictions == y_test) / len(y_test)

To get clearer intuition about predicion, let's look at those X, that should be labeled to some flower

In [ ]:
y_test


Here you can traverse the tree above by yourself and make sure that prediction works

In [ ]:
X_test[1]

In [ ]:
clf.predict([X_test[1]])

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
### Finally, it is your turn to write such classifier by yourself!

####  Gini impurity

Decision trees use the concept of Gini impurity to describe how “pure” a node is. A node is pure (G = 0) if all its samples belong to the same class, while a node with many samples from many different classes will have a Gini closer to 1.

$G = 1 - \sum_{k=1}^{n}p_{k}^2$

For example, if a node contains five samples, with two belonging to the first class (first flower), two of class 2, one of class 3 and none of class 4, then

$G = 1 - (\frac{2}{5})^2 - (\frac{2}{5})^2 - (\frac{1}{5})^2 = 0.64$

#### Remarks 
- We recommend using additional functions in `DecisionTreeClassifier` class, to make the implementation process easier.
- [use this hint](https://arc.net/l/quote/pqvyjqei)

In [ ]:
a = [1,  2,  1,  1,   4, 43, 342, 2]
b = [11, 2,  4,  554, 3, 6,  6,   2]
a = np.array(a)
b = np.array(b)
sum(a == b) / len(b)

In [46]:
class Node:
    def __init__(self, X: npt.NDArray, y: npt.NDArray, feature_index: int=0, threshold:
    float=0, left=None, right=None, value=None):
        """
        :param X: numpy array of form [[feature1,feature2, ... featureN], ...] (i.e. [[1.5, 5.4, 3.2, 9.8] , ...] for case with iris d.s.)
        :param y: numpy array of from [class1, class2, ...] (i.e. [0,1,1,2,1,0,...] for case with iris d.s.)
        """

        self.value = value
        self.X = X
        self.y = y
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right

In [47]:
class DecisionTreeClassifier:
    def __init__(self, max_depth: int=5) -> None:
        self.max_depth = max_depth
        self.tree = None
        self.number_of_classes = None

    def fit(self, X: npt.NDArray, y: npt.NDArray) -> None:
        """
        Basically, function that performs all the training (building of a tree)
        We recommend to use it as a wrapper of recursive building function
        """
        self.number_of_classes = np.unique(y).size
        self.tree = self.build_tree(X, y)

    def build_tree(self, X: npt.NDArray, y: npt.NDArray, depth=0):
        sample_size, feature_size = X.shape
        # print(sample_size, feature_size)
        if sample_size > 1 and depth <= self.max_depth:
            best_split = self.get_best_split(X, y, feature_size)
            if best_split['IG'] > 0:
                l_tree = self.build_tree(best_split['left_split'][:, :-1], best_split[
                    'left_split'][:, -1], depth + 1)
                r_tree = self.build_tree(best_split['right_split'][:, :-1], best_split[
                    'right_split'][:, -1], depth + 1)
                return Node(X, y, best_split['index'], best_split['threshold'], l_tree, r_tree)

        return Node(X, y)

    def get_best_split(self, X: npt.NDArray, y: npt.NDArray, feature_size: int)\
            -> dict:
        best_split = {}
        max_info_gain = float('-inf')

        for feature_index in range(feature_size):
            feature_values = X[:, feature_index]
            unique_thresholds = np.unique(feature_values)
            for threshold in unique_thresholds:
                data = np.c_[X, y]
                left_data, right_data = self.split(data, feature_index, threshold)
                if len(left_data) <= 0 or len(right_data) <= 0:
                    continue
                left_y, right_y = left_data[:, -1], right_data[:, -1]
                info_gain = self.calc_info_gain(y, left_y, right_y)

                if info_gain > max_info_gain:
                    best_split = {
                        'index': feature_index,
                        'threshold': threshold,
                        'left_split': left_data,
                        'right_split': right_data,
                        'IG': info_gain
                    }
                    max_info_gain = info_gain
        return best_split

    def calc_info_gain(self, y, left_y, right_y):
        left_weight = len(left_y) / len(y)
        right_weight = len(right_y) / len(y)
        return self.calc_entropy(y) - (left_weight * self.calc_entropy(left_y) + right_weight * self.calc_entropy(right_y))

    @staticmethod
    def calc_entropy(y):
        y = np.unique(y, return_counts=True)[1]/len(y)
        return np.sum(-y * np.log2(y))

    @staticmethod
    def split(data, feature_index, threshold):
        dataset_left = np.array([row for row in data if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in data if row[feature_index] > threshold])
        return dataset_left, dataset_right

    def predict(self, X_test: npt.NDArray):
        """
        Traverse the tree while there is a child
        and return the predicted class for it
        """
        return np.array([self.make_prediction(x, self.tree) for x in X_test])

    @staticmethod
    def calc_val(y):
        y = list(y)
        return max(y, key=y.count)

    def make_prediction(self, x, tree):
        if tree.left is None:
            return self.calc_val(tree.y)
        if x[tree.feature_index] <= tree.threshold:
            return self.make_prediction(x, tree.left)
        return self.make_prediction(x, tree.right)

In [48]:

def evaluate(X_test: npt.NDArray, y_test: npt.NDArray):
    """
    Returns accuracy of the model (ratio of right guesses to the number of samples)
    """
    print(f'The tree acuracy is: {sum(X_test == y_test) / len(X_test) * 100:.2f}%')

In [107]:
# X, y = iris.data, iris.target
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

X, X_test, y, y_test = train_test_split(X, y, test_size= 0.20)

In [109]:

tree = DecisionTreeClassifier(max_depth=10)
tree.fit(X, y)
evaluate(tree.predict(X_test), y_test)

The tree acuracy is: 92.11%
